In [1]:
import tensorflow as tf
import time

In [2]:
tf.__version__

'2.13.0'

Prefetch

In [4]:
class FileDataset(tf.data.Dataset):
    def read_file_in_batches(num_samples):
        # opening the file
        time.sleep(0.03)
        
        for sample_idx in range(num_samples):
            # reading data (line,record) from the file
            time.sleep(0.015)
            
            yield (sample_idx)
            
        def __new__(cls,num_samples = 3):
            return tf.data.Dataset.from_generator(
                cls.read_file_in_batches,
                output_signature=tf.TensorSpec(shape=(1,),dtype=tf.int64),
                args=(num_samples,)
            )

In [5]:
def benchmark(dataset,num_epochs=2):
    for epoch_num in range(num_epochs):
        for sample in dataset:
            # performing a training step
            time.sleep(0.01)

In [7]:
%%timeit
benchmark(FileDataset())

TypeError: Can't instantiate abstract class FileDataset with abstract methods _inputs, element_spec

In [8]:
%%timeit
benchmark(FileDataset().prefetch(1))

TypeError: Can't instantiate abstract class FileDataset with abstract methods _inputs, element_spec

In [9]:
%%timeit
benchmark(FileDataset().prefetch(tf.data.AUTOTUNE))

TypeError: Can't instantiate abstract class FileDataset with abstract methods _inputs, element_spec

Cache

In [10]:
dataset = tf.data.Dataset.range(5)
dataset = dataset.map(lambda x: x**2)
dataset = dataset.cache("mycache.txt")
# The first time reading through the data will generate the data using
# `range` and `map`.
list(dataset.as_numpy_iterator())

[0, 1, 4, 9, 16]

In [11]:
# Subsequent iterations read from the cache.
list(dataset.as_numpy_iterator())

[0, 1, 4, 9, 16]

In [13]:
def mapped_function(s):
    # Do some hard pre-processing
    tf.py_function(lambda: time.sleep(0.03), [], ())
    return s

In [14]:
%%timeit -r1 -n1
benchmark(FileDataset().map(mapped_function), 5)

TypeError: Can't instantiate abstract class FileDataset with abstract methods _inputs, element_spec

In [15]:
%%timeit -r1 -n1
benchmark(FileDataset().map(mapped_function).cache(), 5)

TypeError: Can't instantiate abstract class FileDataset with abstract methods _inputs, element_spec